**注意**：文档中标注 * 的章节不属于作业要求，作为补充内容供同学们参考。

# 1. Package 加载与设置

运行前可以使用 `env_config.sh` 脚本进行环境配置，具体可以看脚本文件中的注释说明。

In [1]:
from IPython import get_ipython  # type: ignore
import base64

ipython = get_ipython()
assert ipython is not None
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

import torch
from tqdm.auto import tqdm
import plotly.express as px
import pandas as pd

torch.set_grad_enabled(False)

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")


Device: cuda


In [2]:
import warnings

# 屏蔽来自 eindex.indexing 的特定弃用告警（PyTorch 2.9 将改变行为）
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    message=r"Using a non-tuple sequence for multidimensional indexing is deprecated.*",
    module=r"eindex\.indexing",
)


# 2. 加载模型和 SAE 模块

SAELens 工具包提供了已经预训练好的 SAE 模块，大部分模型都支持在 Neuronpedia 平台上直接调用。

这里我们先查看一下针对 gemma-2-2b 模型的相关 SAE 模块

In [3]:
from sae_lens.loading.pretrained_saes_directory import get_pretrained_saes_directory

# TODO: Make this nicer.
df = pd.DataFrame.from_records(
    {k: v.__dict__ for k, v in get_pretrained_saes_directory().items()}
).T
df.drop(
    columns=[
        "expected_var_explained",
        "expected_l0",
        "config_overrides",
        "conversion_func",
    ],
    inplace=True,
)
# 查找包含 gemmascope 的行，不区分大小写，gemmascope 是 DeepMind 团队针对 gemma 模型训练的一系列 SAE
df = df[df["repo_id"].str.contains("gemma-scope-2b-pt", case=False)]
df

release  \
gemma-scope-2b-pt-att                      gemma-scope-2b-pt-att   
gemma-scope-2b-pt-att-canonical  gemma-scope-2b-pt-att-canonical   
gemma-scope-2b-pt-mlp                      gemma-scope-2b-pt-mlp   
gemma-scope-2b-pt-mlp-canonical  gemma-scope-2b-pt-mlp-canonical   
gemma-scope-2b-pt-res                      gemma-scope-2b-pt-res   
gemma-scope-2b-pt-res-canonical  gemma-scope-2b-pt-res-canonical   
gemma-scope-2b-pt-transcoders      gemma-scope-2b-pt-transcoders   

                                                              repo_id  \
gemma-scope-2b-pt-att                    google/gemma-scope-2b-pt-att   
gemma-scope-2b-pt-att-canonical          google/gemma-scope-2b-pt-att   
gemma-scope-2b-pt-mlp                    google/gemma-scope-2b-pt-mlp   
gemma-scope-2b-pt-mlp-canonical          google/gemma-scope-2b-pt-mlp   
gemma-scope-2b-pt-res                    google/gemma-scope-2b-pt-res   
gemma-scope-2b-pt-res-canonical          google/gemma-scope-2b-pt-res   
gemma-scope-2b-pt-transcoders    google/gemma-scope-2b-pt-transcoders   

                                      model  \
gemma-scope-2b-pt-att            gemma-2-2b   
gemma-scope-2b-pt-att-canonical  gemma-2-2b   
gemma-scope-2b-pt-mlp            gemma-2-2b   
gemma-scope-2b-pt-mlp-canonical  gemma-2-2b   
gemma-scope-2b-pt-res            gemma-2-2b   
gemma-scope-2b-pt-res-canonical  gemma-2-2b   
gemma-scope-2b-pt-transcoders    gemma-2-2b   

                                                                          saes_map  \
gemma-scope-2b-pt-att            {'layer_0/width_16k/average_l0_104': 'layer_0/...   
gemma-scope-2b-pt-att-canonical  {'layer_0/width_16k/canonical': 'layer_0/width...   
gemma-scope-2b-pt-mlp            {'layer_0/width_16k/average_l0_119': 'layer_0/...   
gemma-scope-2b-pt-mlp-canonical  {'layer_0/width_16k/canonical': 'layer_0/width...   
gemma-scope-2b-pt-res            {'embedding/width_4k/average_l0_6': 'embedding...   
gemma-scope-2b-pt-res-canonical  {'layer_0/width_16k/canonical': 'layer_0/width...   
gemma-scope-2b-pt-transcoders    {'layer_0/width_16k/average_l0_76': 'layer_0/w...   

                                                                    neuronpedia_id  
gemma-scope-2b-pt-att            {'layer_0/width_16k/average_l0_104': None, 'la...  
gemma-scope-2b-pt-att-canonical  {'layer_0/width_16k/canonical': 'gemma-2-2b/0-...  
gemma-scope-2b-pt-mlp            {'layer_0/width_16k/average_l0_119': None, 'la...  
gemma-scope-2b-pt-mlp-canonical  {'layer_0/width_16k/canonical': 'gemma-2-2b/0-...  
gemma-scope-2b-pt-res            {'embedding/width_4k/average_l0_6': None, 'emb...  
gemma-scope-2b-pt-res-canonical  {'layer_0/width_16k/canonical': 'gemma-2-2b/0-...  
gemma-scope-2b-pt-transcoders    {'layer_0/width_16k/average_l0_76': 'gemma-2-2...

查看 gemma-scope-2b-pt-res-canonical 对应的 SAE 模块

可以看到，每一层都对应有不同宽度的SAE，如 16k, 32k，65k 等

In [4]:
print("SAEs in the gemma-2b Resid Pre release")
for k, v in df.loc[df.release == "gemma-scope-2b-pt-res-canonical", "saes_map"].values[0].items():
    print(f"SAE id: {k} for hook point: {v}")

SAEs in the gemma-2b Resid Pre release
SAE id: layer_0/width_16k/canonical for hook point: layer_0/width_16k/average_l0_105
SAE id: layer_1/width_16k/canonical for hook point: layer_1/width_16k/average_l0_102
SAE id: layer_2/width_16k/canonical for hook point: layer_2/width_16k/average_l0_141
SAE id: layer_3/width_16k/canonical for hook point: layer_3/width_16k/average_l0_59
SAE id: layer_4/width_16k/canonical for hook point: layer_4/width_16k/average_l0_124
SAE id: layer_5/width_16k/canonical for hook point: layer_5/width_16k/average_l0_68
SAE id: layer_6/width_16k/canonical for hook point: layer_6/width_16k/average_l0_70
SAE id: layer_7/width_16k/canonical for hook point: layer_7/width_16k/average_l0_69
SAE id: layer_8/width_16k/canonical for hook point: layer_8/width_16k/average_l0_71
SAE id: layer_9/width_16k/canonical for hook point: layer_9/width_16k/average_l0_73
SAE id: layer_10/width_16k/canonical for hook point: layer_10/width_16k/average_l0_77
SAE id: layer_11/width_16k/cano

加载模型和每层对应的 SAE 模块，这里在每层上统一选择宽度为 16k 的 SAE 模块

注意，为了节约显存，model 是直接加载到 GPU 上的，而每层的 SAE 是加载到 CPU 上的，后续在使用的时候再转移到 GPU 上。

In [5]:
# from transformer_lens import HookedTransformer
from sae_lens import SAE, HookedSAETransformer

model = HookedSAETransformer.from_pretrained("gemma-2-2b", device=device, center_unembed=False)

sae_per_layer = {}
for i in tqdm(range(26)):
    sae_per_layer[f"layer_{i}"] = SAE.from_pretrained(
        release="gemma-scope-2b-pt-res-canonical",  # <- Release name
        sae_id=f"layer_{i}/width_16k/canonical",  # <- SAE id (not always a hook point!)
        device='cpu',
    )
    sae_per_layer[f"layer_{i}"].fold_W_dec_norm()

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


  0%|          | 0/26 [00:00<?, ?it/s]

可以连接到 neuronpedia 平台查看指定模型、指定SAE模块的特征（feature）

例如这里，我们可以查看 gemma-2-2b 模型，第8层的 16k 宽度SAE，第 12869 个 feature

In [7]:
from IPython.display import IFrame

# get a random feature from the SAE
# feature_idx = torch.randint(0, sae.cfg.d_sae, (1,)).item()

def get_dashboard_html(sae_release="gemma-2-2b", sae_id="8-gemmascope-res-16k", feature_idx=0):
    html_template = "https://neuronpedia.org/{}/{}/{}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"
    return html_template.format(sae_release, sae_id, feature_idx)


html = get_dashboard_html(
    sae_release="gemma-2-2b", sae_id="20-gemmascope-res-16k", feature_idx=14838
)

IFrame(html, width=1200, height=500)

# 3. 特征检索*

也可以对 neuronpedia 上自动生成的 feature description 进行检索。

In [8]:
import requests

url = "https://www.neuronpedia.org/api/explanation/export?modelId=gemma-2-2b&saeId=5-gemmascope-res-16k"
headers = {"Content-Type": "application/json"}

response = requests.get(url, headers=headers)

# convert to pandas
data = response.json()
explanations_df = pd.DataFrame(data)
# rename index to "feature"
explanations_df.rename(columns={"index": "feature"}, inplace=True)
# explanations_df["feature"] = explanations_df["feature"].astype(int)
explanations_df["description"] = explanations_df["description"].apply(
    lambda x: x.lower()
)
explanations_df

modelId                 layer feature  \
0      gemma-2-2b  5-gemmascope-res-16k    7619   
1      gemma-2-2b  5-gemmascope-res-16k     274   
2      gemma-2-2b  5-gemmascope-res-16k    4217   
3      gemma-2-2b  5-gemmascope-res-16k    5842   
4      gemma-2-2b  5-gemmascope-res-16k     791   
...           ...                   ...     ...   
16576  gemma-2-2b  5-gemmascope-res-16k    2586   
16577  gemma-2-2b  5-gemmascope-res-16k      40   
16578  gemma-2-2b  5-gemmascope-res-16k    6449   
16579  gemma-2-2b  5-gemmascope-res-16k    5358   
16580  gemma-2-2b  5-gemmascope-res-16k   11301   

                                             description explanationModelName  \
0                                   hashtags in the text          gpt-4o-mini   
1                                       urls within text          gpt-4o-mini   
2                                        dates and times          gpt-4o-mini   
3                            html tags and code snippets          gpt-4o-mini   
4       sentences that express strong emotions or piv...          gpt-4o-mini   
...                                                  ...                  ...   
16576                   php and template syntax elements          gpt-4o-mini   
16577                c-style pointer and function syntax          gpt-4o-mini   
16578                         references to web browsers          gpt-4o-mini   
16579                     terms related to organizations          gpt-4o-mini   
16580                        equal signs in the document          gpt-4o-mini   

                 typeName  
0      oai_token-act-pair  
1      oai_token-act-pair  
2      oai_token-act-pair  
3      oai_token-act-pair  
4      oai_token-act-pair  
...                   ...  
16576  oai_token-act-pair  
16577  oai_token-act-pair  
16578  oai_token-act-pair  
16579  oai_token-act-pair  
16580  oai_token-act-pair  

[16581 rows x 6 columns]

In [9]:
dog_features = explanations_df.loc[explanations_df.description.str.contains("dog")]
dog_features

modelId                 layer feature  \
2025  gemma-2-2b  5-gemmascope-res-16k    2001   
2706  gemma-2-2b  5-gemmascope-res-16k     877   
7088  gemma-2-2b  5-gemmascope-res-16k    7963   
7716  gemma-2-2b  5-gemmascope-res-16k    8845   

                                            description explanationModelName  \
2025   phrases related to restraint and tethering, p...          gpt-4o-mini   
2706                                 references to dogs          gpt-4o-mini   
7088  references to dogs, their behavior, and traini...          gpt-4o-mini   
7716  terms associated with employment, features of ...          gpt-4o-mini   

                typeName  
2025  oai_token-act-pair  
2706  oai_token-act-pair  
7088  oai_token-act-pair  
7716  oai_token-act-pair

# 4. 特征发现

加载 SAE 模块后，我们可以使用自定义的语料库和 Prompt，来探测与指定的概念对应的特征。

首先，我们加载自定义的语料库，并进行 tokenizeation 处理。

In [10]:
import json
corpus = json.loads(open("example_corpus.json").read())
print(corpus.keys())
corpus_sents = []
for key in corpus:
    corpus_sents.extend(corpus[key])

tokens = model.to_tokens(corpus_sents, prepend_bos=True).to(device)
# 去掉 pad 后 concat 到一起
tokens = torch.cat([t[t != model.tokenizer.pad_token_id] for t in tokens], dim=0).unsqueeze(0)
print(tokens.shape)

dict_keys(['dog', 'labrador', 'shepherd', 'corgi', 'husky', 'poodle', 'beagle', 'hound'])
torch.Size([1, 2482])


下面的几个单元格定义了这样几个函数：

- get_target_idx，给定包含目标词语（topic）的句子（prompt），返回该词在分词后的句子（token_prompt）中的索引位置。n_search 表示检索该词语第几次出现。
- collect_cache_per_layer，获取给定句子（prompt）在每层 SAE 模块上的特征激活数据。
- draw_dashboard_local，生成指定特征的 dashboard，在本地存储为一个 html 文件
- sae_dashboard_in_line，将本地的特征 dashboard 绘制到当前的 notebook 中

In [11]:
def get_target_idx(prompt: str, topic: str, token_prompt: list, n_search: int = 1):
    low_prompt = prompt.lower()
    pos = 0
    for idx, _ in enumerate(range(n_search)):
        search_pos = pos + 1 if pos != 0 or idx != 0 else 0
        pos = low_prompt.find(topic, search_pos)
        if pos == -1:
            return -1

    substring_prompt = prompt[:pos]
    space_count = substring_prompt.count(" ")

    token_space_count = 0
    target_idx = -1
    for idx, token in enumerate(token_prompt):
        if " " in token:
            token_space_count += 1
        if token_space_count == space_count:
            # 如果 token_prompt 中的下一个 token 不以空格开始，且不是标点，则 token_idx += 1，循环继续
            while True:
                next_idx = idx + 1
                if next_idx >= len(token_prompt):
                    break
                next_token = token_prompt[next_idx]
                if not next_token.startswith(" ") and not all(
                    char in '.,!?;:"\'()[]{}' for char in next_token
                ):
                    idx += 1
                else:
                    break
            target_idx = idx
            break
    return target_idx

In [12]:
def collect_cache_per_layer(prompt):
    cache_per_layer = {}
    for l in tqdm(range(26), desc="Collecting Cache", leave=False):
        sae = sae_per_layer[f"layer_{l}"].to(device)
        # hooked SAE Transformer will enable us to get the feature activations from the SAE
        _, cache = model.run_with_cache_with_saes(prompt, saes=[sae])
        cache_cpu = {k: v.cpu() for k, v in cache.items()}
        cache_per_layer[f"layer_{l}"] = cache_cpu
        sae.to("cpu")
        del cache
        torch.cuda.empty_cache()
    return cache_per_layer

In [13]:
# from sae_dashboard.sae_vis_data import SaeVisConfig, SaeVisData
import os
from sae_dashboard.sae_vis_runner import SaeVisRunner, SaeVisConfig
from sae_dashboard.data_writing_fns import save_feature_centric_vis

def draw_dashboard_local(topic, layer, inds):
    sae = sae_per_layer[f"layer_{layer}"].to(device)

    dashboard_config = SaeVisConfig(
        hook_point=sae.cfg.metadata.hook_name,
        features=inds,
        minibatch_size_features=64,
        minibatch_size_tokens=256,
        device=device,
        dtype='float16'
    )

    dashboard_data = SaeVisRunner(dashboard_config).run(encoder=sae, model=model, tokens=tokens)

    os.makedirs(f"dashboard_files/{topic}_layer_{layer}", exist_ok=True)

    save_feature_centric_vis(
        sae_vis_data=dashboard_data,
        feature_idx=inds,
        filename=f"dashboard_files/{topic}_layer_{layer}/vis.html",
        separate_files=True
    )
    sae.to("cpu")
    torch.cuda.empty_cache()

In [14]:
def sae_dashboard_in_line(topic, layer, inds, vals):
    for ind, val in zip(inds, vals):
        file_name = f"dashboard_files/{topic}_layer_{layer}/vis_feature_{int(ind)}.html"
        html_content = open(file_name).read()
        encoded_html = base64.b64encode(html_content.encode("utf-8")).decode("utf-8")
        url = "data:text/html;base64," + encoded_html
        print(f"layer {layer}, feature {ind}, value {val:.4f}, file, {file_name}")
        display(IFrame(url, width=1200, height=500))

对于给定的词语，可以使用上面的函数，获取句子每层的激活特征。

此外，通过 target_idx，可以看到我们关心的词语 dog 在这个句子中是第 4 个token。

In [15]:
topic = "dog"
# prompt = " ".join([corpus[topic][0]] * 2)
prompt = corpus[topic][0]
token_prompt = model.to_str_tokens(prompt, prepend_bos=True)
target_idx = get_target_idx(prompt, topic, token_prompt, n_search=1)
print(token_prompt)
print(target_idx)
print(token_prompt[:target_idx])
cache_per_layer = collect_cache_per_layer(prompt)


['<bos>', 'I', ' take', ' my', ' dog', ' for', ' a', ' walk', ' in', ' the', ' park', ' every', ' morning', '.']
4
['<bos>', 'I', ' take', ' my']


根据收集到的激活数据，我们可以分析指定层、指定位置的前 k 个特征。

在这里，我们观察第 20 层，在 dog 这个词上激活值最高的前 5 个特征。

In [16]:
from IPython.display import IFrame

layer = 20
top_k = 5

# hover over lines to see the Feature ID.
px.line(
    cache_per_layer[f"layer_{layer}"][f"blocks.{layer}.hook_resid_post.hook_sae_acts_post"][0, target_idx, :].cpu().numpy(),
    title="Feature activations at the final token position",
    labels={"index": "Feature", "value": "Activation"},
).show()

vals, inds = torch.topk(
    cache_per_layer[f"layer_{layer}"][f"blocks.{layer}.hook_resid_post.hook_sae_acts_post"][0, target_idx, :], top_k
)
draw_dashboard_local(topic, layer, inds.tolist())
sae_dashboard_in_line(topic, layer, inds.tolist(), vals.tolist())

layer 20, feature 12082, value 91.0775, file, dashboard_files/dog_layer_20/vis_feature_12082.html


layer 20, feature 3717, value 70.5122, file, dashboard_files/dog_layer_20/vis_feature_3717.html


layer 20, feature 14838, value 61.5894, file, dashboard_files/dog_layer_20/vis_feature_14838.html


layer 20, feature 6631, value 51.8808, file, dashboard_files/dog_layer_20/vis_feature_6631.html


layer 20, feature 11640, value 47.9488, file, dashboard_files/dog_layer_20/vis_feature_11640.html


# 5. 特征验证（Steering）

上面我们通过观察词语在 SAE 中的特征激活情况，找到了和这个词语最相关的特征。

通过操纵这些特征，我们可以改变模型的行为。比如，通过增强和 dog 相关的特征，让模型的输出中包含 dog 这个词。

In [17]:
import requests
import numpy as np
from pprint import pp

seed = np.random.randint(0, 10000)
print(f"Using seed: {seed}")
# labrador seed: 31

url = "https://www.neuronpedia.org/api/steer"

payload = {
    "prompt": 'An interesting animal is',
    # "prompt": 'An interesting dog is',
    "modelId": "gemma-2-2b",
    "features": [
        {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 12082, "strength": 100}, # dog, seed 16
        {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 14838, "strength": 30}, # dog
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 12100, "strength": 100}, # labrador, seed 31
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 5852, "strength": 500}, # labrador
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 13751, "strength": 200}, # labrador
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 15824, "strength": 300}, # shepherd, seed 31
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 6596, "strength": 300}, # shepherd
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 9145, "strength": 300}, # shepherd
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 863, "strength": 150}, # corgi, #cor
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 24, "strength": 150}, # corgi
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 2801, "strength": 100}, # corgi
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 12082, "strength": -200}, # corgi, after cor
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 13747, "strength": 82}, # poodle
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 6631, "strength": 60}, # poodle
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 12082, "strength": 60}, # poodle
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 15509, "strength": 40}, # poodle
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 11355, "strength": 40}, # poodle
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 13761, "strength": 40}, # poodle
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 1692, "strength": 40}, # poodle
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 9768, "strength": 30}, # poodle
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 3942, "strength": 30}, # poodle
        # {"modelId": "gemma-2-2b", "layer": "20-gemmascope-res-16k", "index": 15361, "strength": 20}, # poodle
    ],
    "temperature": 1,
    "n_tokens": 64,
    "freq_penalty": 1,
    "seed": 16,
    "strength_multiplier": 1,
    "steer_method": "SIMPLE_ADDITIVE"
    # "steer_method": "ORTHOGONAL_DECOMP"
}
headers = {"Content-Type": "application/json"}

response = requests.post(url, json=payload, headers=headers)
pp(response.json())

Using seed: 2732
{'STEERED': 'An interesting animal is the typical "r-dog" who would '
            'immediately do any trick for food. But he will happily ignore '
            'commands and treats when no dog \u200b\u200bis around - not a '
            'great recall with separation anxiety problem he needs to be '
            'leashed and the people who own them are occasionally used as a '
            'treat. .\n'
            '\n'
            'When she was ',
 'DEFAULT': 'An interesting animal is an octopus.\n'
            "And here's another! (I wanted to do the others in pairs, but I "
            'already had one too!)\n'
            'Here I attempted to capture a pose with movement, without giving '
            'too much details.\n'
            '\n'
            'Hope the people who looked at the other OC will like this!\n'
            '\n'
            '\n'
            'The next one would be',
 'steeredLogProbs': [{'token': ' the',
                      'logprob': -0.83984375,
    

# 6. 通过 Neuronpedia 平台进行特征观察*
通过 neuronpedia 平台，我们可以验证这些特征在更多语料上的激活情况。

In [18]:
# let's print the top 5 features and how much they fired
vals, inds = torch.topk(
    cache_per_layer[f"layer_{layer}"][f"blocks.{layer}.hook_resid_post.hook_sae_acts_post"][0, target_idx, :], top_k
)
for val, ind in zip(vals, inds):
    print(f"Feature {ind} fired {val:.2f}")
    html = get_dashboard_html(
        sae_release="gemma-2-2b", sae_id=f"{layer}-gemmascope-res-16k", feature_idx=int(ind.item())
    )
    display(IFrame(html, width=1200, height=300))

Feature 12082 fired 91.08


Feature 3717 fired 70.51


Feature 14838 fired 61.59


Feature 6631 fired 51.88


Feature 11640 fired 47.95
